# [Classification trials](#classification-trials)

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from datetime import datetime
from glob import glob
from io import StringIO
from IPython.display import display
from os.path import getctime
from pathlib import Path
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from azure.storage.blob import BlockBlobService
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
%aimport src.model_runner
from src.model_runner import sklearn_trials

%aimport src.pipe_runner
from src.pipe_runner import (
    boosting_trials,
    get_best_pipe_by_model_name,
    get_best_naive_pipe_by_strategy_name,
    get_preds_probas,
    run_boosted_tuning,
    get_feature_permutation_importances,
)

%aimport src.visualization_helpers
from src.visualization_helpers import (
    plot_horiz_bar,
    plot_horizontal_box_plots,
    show_confusion_matrix,
    show_FeatureImportances,
)

%aimport src.inspection_helpers
from src.inspection_helpers import get_preds_probas

%aimport src.feature_helpers
from src.feature_helpers import (
    apply_over_under_sampling,
    check_mapped_column_unique_values,
    replace_col_values,
)

%aimport src.metrics_helpers
from src.metrics_helpers import my_eval_metric

%aimport src.export_outputs
from src.export_outputs import (
    export_experiment_results,
    export_mapping_data,
)

In [ ]:
SMALL_SIZE = 26
MEDIUM_SIZE = 28
BIGGER_SIZE = 30
plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["axes.facecolor"] = "white"
sns.set_style("darkgrid", {"legend.frameon": False})
sns.set_context("talk", font_scale=0.95, rc={"lines.linewidth": 2.5})

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Load joined data](#load-joined-data)
3. [Exploratory Data Analysis and Feature Engineering](#exploratory-data-analysis-and-feature-engineering)
4. [Re-grouping categorical features](#re-grouping-categorical-features)
5. [Pre-processing categorical features](#pre-processing-categorical-features)
6. [Generate training and testing splits](#generate-training-and-testing-splits)
7. [Pre-processing numerical features](#pre-processing-numerical-features)
8. [Experiments in modeling](#experiments-in-modeling)
9. [Gradient Boosting trials](#gradient-boosting-trials)
10. [Plot model comparison](#plot-model-comparison)
12. [Export experiment outputs](#export-experiment-outputs)
13. [Export data for mapping](#export-data-for-mapping)
13. [Examine feature importances](#examine-feature-importances)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we will experiment with classification models on the joined Chicago crime listings-weather-demographics data in `data/processed/all_joined__mmddyyyy_HHMMss.csv`

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

We'll define below the variables and helper functions that are to be used throughout the code.

In [ ]:
data_dir_path = str(Path().cwd() / "data" / "processed")
dash_data_dir_path = str(Path().cwd() / "aci-dash" / "app" / "data" / "processed")
cloud_data = "no"
figs_dir_path = str(Path().cwd() / "reports" / "figures")
d_mapping_specs = {
    "choro": [
        ["primary_type", "district"],
        {
            "arrest": ["sum"],
            "datetime": ["count"],
            "total_population": ["sum"],
            "median_household_value": ["mean"],
            "median_household_income": ["mean"],
            "probability_of_max_class": ["mean"],
        },
        str(Path(dash_data_dir_path) / "choro_mapping_inputs.csv"),
    ],
    "heatmap": [
        ["primary_type", "day", "month"],
        {
            "arrest": ["sum"],
            "datetime": ["count"],
            "TAVG": ["mean"],
            "SNOW": ["mean"],
            "probability_of_max_class": ["mean"],
        },
        str(Path(dash_data_dir_path) / "heat_mapping_inputs.csv"),
    ],
}

experiment_summary_data_path = str(
    Path(dash_data_dir_path)
    / f"experiment_summary_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv"
)
experiment_all_cv_data_path = str(
    Path(data_dir_path)
    / f"experiment_all_cv_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv"
)

scoring_metric = "accuracy"
n_folds = 5
target = "primary_type"
top_n_features_to_visualize = 10

use_boosting_classifiers = False
boosting_classifier_names = ["LGBMClassifier", "XGBClassifier"]

years_wanted = [2018, 2019]
months_wanted = [1, 2, 3]
non_location_col_choices = {
    0: ["dayofyear", "hour", "location_description", target],
    1: ["month", "day", "hour", "location_description", target],
    2: ["month", "weekday_name", "hour", "location_description", target],
    3: ["weekofyear", "weekday_name", "hour", "location_description", target],
    4: ["year", "month", "is_weekend", "is_dark", "location_description", target],
}
chosen_non_location_cols = non_location_col_choices[4]
features_to_consolidate = ["location_description", "community_area", "primary_type"]

nums = [
    "latitude",
    "longitude",
    "total_population",
    "median_household_income",
    # "occupied_housing_values",
    "median_household_value",
    "TAVG",
    "AWND",
    "PRCP",
    "SNOW",
]
cats = [
    "district",
    # "fbi_code",
    # "ward",
    # "beat",
    "community_area",  # transformed to Side
    "arrest",
    "domestic",
] + chosen_non_location_cols[:-1]

target_balance = "Unbalanced"  # "under_sampled", "over_sampled" or "Unbalanced"

# Mapping dictionaries
d_comm_area_to_side = {
    "Central": [8, 32, 33],
    "North": [5, 6, 7, 21, 22],
    "Far North": [1, 2, 3, 4, 9, 10, 11, 12, 13, 14, 76, 77],
    "Northwest": [15, 16, 17, 18, 19, 20],
    "West": [23, 24, 25, 26, 27, 28, 29, 30, 31],
    "South": [34, 35, 36, 38, 39, 40, 41, 42, 43, 60, 69],
    "Southwest": [56, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68],
    "Far Southeast": [44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
    "Far Southwest": [70, 71, 72, 73, 74, 75],
}

# # Groupings that were used in most of the development
# d_pri_type = {
#     "A": [
#         "THEFT",
#         "BURGLARY",
#         "ROBBERY",
#         "MOTOR VEHICLE THEFT",
#         "CRIMINAL DAMAGE",
#         "CRIMINAL TRESPASS",
#         "ARSON",
#     ],
#     "B": [
#         "BATTERY",
#         "ASSAULT",
#         "KIDNAPPING",
#         "HOMICIDE",
#         "CRIM SEXUAL ASSAULT",
#         "SEX OFFENSE",
#         "HUMAN TRAFFICKING",
#         "OFFENSE INVOLVING CHILDREN",
#         "STALKING",
#         "INTIMIDATION",
#         "PUBLIC INDECENCY",
#         "PROSTITUTION",
#     ],
#     "D": [
#         "DECEPTIVE PRACTICE",
#         "OTHER OFFENSE",
#         "WEAPONS VIOLATION",
#         "CONCEALED CARRY LICENSE VIOLATION",
#         "PUBLIC PEACE VIOLATION",
#         "OBSCENITY",
#         "INTERFERENCE WITH PUBLIC OFFICER",
#         "LIQUOR LAW VIOLATION",
#         "NARCOTICS",
#         "GAMBLING",
#         "NON-CRIMINAL",
#     ],
# }

# Groupings that are closest possible match to Crime Topic Modeling paper:
# https://link.springer.com/content/pdf/10.1186%2Fs40163-017-0074-0.pdf
d_pri_type = {
    "PROPERTY_DAMAGE": [
        "THEFT",
        "BURGLARY",
        "CRIMINAL TRESPASS",
        "ARSON",  # originally RED
        "MOTOR VEHICLE THEFT",  # originally RED
        "CRIMINAL DAMAGE",  # originally RED
    ],  # A (PURPLE)
    "VIOLENCE_TO_HUMAN": [  # B (YELLOW)
        "BATTERY",
        "ASSAULT",
        "KIDNAPPING",
        "HOMICIDE",
        "CRIM SEXUAL ASSAULT",  # I considered to be more serious human-related crime
        "SEX OFFENSE",  # ? I considered to be more serious human-related crime
        "HUMAN TRAFFICKING",  # ? I considered to be more serious human-related crime
        "PROSTITUTION",  # ? I considered to be more serious human-related crime
        "ROBBERY",
        "INTERFERENCE WITH PUBLIC OFFICER",
        "WEAPONS VIOLATION",
        "CONCEALED CARRY LICENSE VIOLATION",
    ],
    "CRIMINAL_DISTURBANCE": [  # D (GREEN)
        "STALKING",
        "INTIMIDATION",
        # "PUBLIC INDECENCY",
        "OFFENSE INVOLVING CHILDREN",
        "DECEPTIVE PRACTICE",
        "OTHER OFFENSE",  # I considered this to be OTHER MISCELLANEOUS CRIME
        "PUBLIC PEACE VIOLATION",
        "OBSCENITY",
        "LIQUOR LAW VIOLATION",  # ?
        "NARCOTICS",  # ?
        "GAMBLING",  # ?
        "NON-CRIMINAL",  # I considered this to be OTHER MISCELLANEOUS CRIME
    ],
    # "VEHICLE AND PROPERTY DAMAGE": ["ARSON", "MOTOR VEHICLE THEFT", "CRIMINAL DAMAGE",],  # E (RED)
}

d_loc_desc = {
    "Public_Accessible": [
        "STREET",
        "SIDEWALK",
        "ALLEY",
        "PARK PROPERTY",
        "HIGHWAY/EXPRESSWAY",
        "BRIDGE",
        # "GANGWAY",
        "PORCH",
        "HALLWAY",
        "HOTEL",
    ],
    "Dining_Shopping": [
        "RESTAURANT",
        "DEPARTMENT STORE",
        "GROCERY FOOD STORE",
        "SMALL RETAIL STORE",
        "BAR OR TAVERN",
        "TAVERN/LIQUOR STORE",
        "CONVENIENCE STORE",
        "APPLIANCE STORE",
        "PAWN SHOP",
        "RETAIL STORE",
    ],
    "Private_Housing": [
        "RESIDENCE",
        "APARTMENT",
        "RESIDENCE PORCH/HALLWAY",
        "RESIDENTIAL YARD (FRONT/BACK)",
        "RESIDENCE-GARAGE",
        "HOTEL/MOTEL",
        "CHA APARTMENT",
        "DRIVEWAY - RESIDENTIAL",
        "CHA PARKING LOT/GROUNDS",
        "CHA HALLWAY/STAIRWELL/ELEVATOR",
        "HOUSE",
        "KENNEL",
        "BASEMENT",
        "YARD",
    ],
    "Transport": [
        "VEHICLE NON-COMMERCIAL",
        "TAXICAB",
        "VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)",
        "VEHICLE-COMMERCIAL",
        "OTHER RAILROAD PROP / TRAIN DEPOT",
        "OTHER COMMERCIAL TRANSPORTATION",
        "VEHICLE - DELIVERY TRUCK",
        "VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS",
        "VEHICLE-COMMERCIAL - TROLLEY BUS",
    ],
    "Business_Academic": [
        "SCHOOL, PUBLIC, BUILDING",
        "COMMERCIAL / BUSINESS OFFICE",
        "BANK",
        "CURRENCY EXCHANGE",
        "ATM (AUTOMATIC TELLER MACHINE)",
        "SCHOOL, PUBLIC, GROUNDS",
        "SCHOOL, PRIVATE, BUILDING",
        "LIBRARY",
        "MOVIE HOUSE/THEATER",
        "COLLEGE/UNIVERSITY GROUNDS",
        "SCHOOL, PRIVATE, GROUNDS",
        "COLLEGE/UNIVERSITY RESIDENCE HALL",
        "SAVINGS AND LOAN",
        "CLEANING STORE",
        "CREDIT UNION",
        "NEWSSTAND",
    ],
    "CTA": [
        "CTA TRAIN",
        "CTA STATION",
        "CTA PLATFORM",
        "CTA BUS",
        "CTA BUS STOP",
        "CTA GARAGE / OTHER PROPERTY",
        "CTA TRACKS - RIGHT OF WAY",
    ],
    "Medical": [
        "HOSPITAL BUILDING/GROUNDS",
        "DRUG STORE",
        "NURSING HOME/RETIREMENT HOME",
        "MEDICAL/DENTAL OFFICE",
        "DAY CARE CENTER",
        "ANIMAL HOSPITAL",
    ],
    "Government": [
        "POLICE FACILITY/VEH PARKING LOT",
        "GOVERNMENT BUILDING/PROPERTY",
        "FEDERAL BUILDING",
        "JAIL / LOCK-UP FACILITY",
        "FIRE STATION",
        "GOVERNMENT BUILDING",
    ],
    "Religious": ["CHURCH/SYNAGOGUE/PLACE OF WORSHIP", "CEMETARY"],
    "Leisure": [
        "ATHLETIC CLUB",
        "BARBERSHOP",
        "SPORTS ARENA/STADIUM",
        "POOL ROOM",
        "BOWLING ALLEY",
        "COIN OPERATED MACHINE",
        "YMCA",
    ],
    "Manufacturing_Vacant": [
        "VACANT LOT/LAND",
        "CONSTRUCTION SITE",
        "ABANDONED BUILDING",
        "WAREHOUSE",
        "FACTORY/MANUFACTURING BUILDING",
        "VACANT LOT",
        "OTHER",
    ],
    "AutoCareSales": [
        "GAS STATION",
        "CAR WASH",
        "AUTO / BOAT / RV DEALERSHIP",
        "AUTO",
        "BOAT/WATERCRAFT",
        "GARAGE/AUTO REPAIR",
        "PARKING LOT",
        "PARKING LOT/GARAGE(NON.RESID.)",
    ],
    "AIRPORT": [
        "AIRPORT VENDING ESTABLISHMENT",
        "AIRPORT EXTERIOR - SECURE AREA",
        "AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA",
        "AIRPORT PARKING LOT",
        "AIRPORT BUILDING NON-TERMINAL - SECURE AREA",
        "AIRPORT TERMINAL LOWER LEVEL - SECURE AREA",
        "AIRCRAFT",
        "AIRPORT EXTERIOR - NON-SECURE AREA",
        "AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA",
        "AIRPORT TRANSPORTATION SYSTEM (ATS)",
    ],
    "PARKS": ["LAKEFRONT/WATERFRONT/RIVERBANK", "FOREST PRESERVE", "FARM"],
}

model_params = {}
model_params["DummyClassifier__most_frequent"] = {
    "strategy": "most_frequent",
}
model_params["DummyClassifier__uniform"] = {
    "strategy": "uniform",
}
model_params["DummyClassifier__stratified"] = {
    "strategy": "stratified",
}
model_params["LogisticRegression"] = {
    "penalty": "l1",
    "multi_class": "ovr",
    "solver": "liblinear",
    "class_weight": "balanced",
    "max_iter": 5000,
}
model_params["LinearSVC"] = {
    "class_weight": "balanced",
    "max_iter": 1000,
}
model_params["GaussianNB"] = {}
model_params["RandomForestClassifier"] = {
    "n_estimators": 1500,
    "class_weight": "balanced",
    "max_depth": 10,
    "n_jobs": -1,
    "random_state": 0,
}

sklearn_wanted_classifiers = ["LogisticRegression", "GaussianNB"]
dummy_strategies = ["most_frequent", "uniform", "stratified"]

In [ ]:
if cloud_data != "yes":
    data_dir_path = Path(data_dir_path)
    comb_data_filepath = max(
        glob(str(data_dir_path / "all_joined__*.csv")), key=getctime
    )
    save_pref = True
else:
    az_storage_container_name = "myconedesx7"
    blob_service = BlockBlobService(
        account_name=os.environ.get("AZURE_STORAGE_ACCOUNT"),
        account_key=os.environ.get("AZURE_STORAGE_KEY"),
    )
    comb_data_filepath = StringIO(
        blob_service.get_blob_to_text(
            container_name=az_storage_container_name, blob_name="blobedesz1"
        ).content
    )
    save_pref = False
(
    figs_dir_path,
    # joined_data_file_path,
    d_mapping_specs["choro"][-1],
    d_mapping_specs["heatmap"][-1],
    experiment_summary_data_path,
    experiment_all_cv_data_path,
) = [
    Path(figs_dir_path),
    # Path(joined_data_file_path),
    Path(d_mapping_specs["choro"][-1]),
    Path(d_mapping_specs["heatmap"][-1]),
    Path(experiment_summary_data_path),
    Path(experiment_all_cv_data_path),
]
sk_folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1000)

<a id="load-joined-data"></a>

## 2. [Load joined data](#load-joined-data)

We'll start by loading the joined data into a `DataFrame`

In [ ]:
df = pd.read_csv(comb_data_filepath, index_col=0)

Next, we'll filter the data to only include the
- years 2018 and 2019
- months of December, January, February and March

In [ ]:
df = df[(df["year"].isin(years_wanted)) & (df["month"].isin(months_wanted))]

<a id="exploratory-data-analysis-and-feature-engineering"></a>

## 3. [Exploratory Data Analysis and Feature Engineering](#exploratory-data-analysis-and-feature-engineering)

We'll count the unique of the class labels and various categorical columns in the joined data
- columns that were included in the crime listings data
  - `district`
  - `ward`
  - `community_area`
  - `fbi_code`
  - `description`
  - `location_description`
- columns that were appended from the [Chicago open data portal](https://data.cityofchicago.org/browse?limitTo=datasets)
  - [`neighbourhood`](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/9wp7-iasj)

In [ ]:
display(
    pd.DataFrame(
        df[
            [
                "district",
                "ward",
                "community_area",
                "fbi_code",
                "description",
                "beat",
                "location_description",
                "neighbourhood",
                "primary_type",
            ]
        ].nunique(),
        columns=["num_unique_values"],
    ).sort_values(by=["num_unique_values"], ascending=True)
)

Thinking ahead to how we would want to visualize this data, six of these features (excluding those containing description information - `location_description` and `description`) may be used for this purpose. We can find boundary files for the following features on the city's open data portal and would allow us to visualize crime by each of these grographical areas within the city, by coloring within the boundaries based on occurrences or arrests there
- [`neighborhood`](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/bbvz-uum9)
- [`district` (or sector) and `beat`](https://data.cityofchicago.org/Public-Safety/Boundaries-Police-Beats-current-/aerh-rz74)
- [`community_area`](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6#Export)

Additionally, we can [map **Side** of Chicago to its Community Areas](https://en.wikipedia.org/wiki/Community_areas_in_Chicago#Community_areas_by_side) by the community area identifier. Unfortunately, a bounddary file for side of the city is not available. This means we can map community areas but visualize crime by side of the city, which would potentially result in multiple community ares being assigned the same color since they fall within the same side.

Boundary files are not available for `ward` so, analogous to `Side`, we would map one of the other types of geographic areas (eg. `neighborhood`s) but color by `ward` such that multiple areas (eg. multiple `neighborhood`s) would have the same color since they fall within the same `ward`.

Before using any of the two `description`-based categorical features (`X`), we'll try to transform them by grouping them further. We'll do the same for the `primary_type` (these are the class labels, `y`). The benefit of applying such a transformation are
- features (`X`)
  - fewer one-hot encoded features will be eventually be produced
- class labels (`y`)
  - we will not be trying to predict class labels that occur infrequently throughout the dataset

<a id="re-grouping-categorical-features"></a>

## 4. [Re-grouping categorical features](#re-grouping-categorical-features)

We'll start by discussing three of the dictionaries above that will be used to combine categorical features (`X`s), including the class labels (`y`s), into larger groups

In each dictionary, the key represents the new larger group name to be assigned.

The dictionary to preform this grouping for the class labels is `d_pri_type` and two versions of this dictionary are shown. In the first (commented) version, a large number of groupings are created. This results in some groups having a small number of observations, while others have a large number. This making it [difficult for the classifier to predict labels for both groups](https://www.researchgate.net/post/Machine_learning_if_proportion_of_number_of_cases_in_different_class_in_training_set_matters).

In [ ]:
for c, d in zip(
    features_to_consolidate, [d_loc_desc, d_comm_area_to_side, d_pri_type],
):
    print(
        c, df[c].value_counts().shape[0], len([i for q in list(d.values()) for i in q])
    )
    print(
        list(
            set(df[c].value_counts().index.tolist())
            - set([i for q in list(d.values()) for i in q])
        )
    )

Before replacing categories by larger groups using the mapping dictionaries, we'll use a helper function to verify that our mapping dictionary replaces all the unique values in the corresponding column of the `DataFrame` containing the data

Next, we'll run this helper function over all features that we want to consolidate into larger groups in order to check that we have included all unique values in the feature in our mapping dictionary's values

In [ ]:
col_replacements = dict(
    list(zip(features_to_consolidate, [d_loc_desc, d_comm_area_to_side, d_pri_type]))
)

for col_to_check, map_dict in zip(
    features_to_consolidate, [d_loc_desc, d_comm_area_to_side, d_pri_type]
):
    check_mapped_column_unique_values(
        df=df, colname=col_to_check, mapping_dict=map_dict
    )

Next, we'll use a helper function to preform this replacement. The function will map lists in the above dictionaries' keys to `DataFrame` columns. This will replace the current value of the column with the name of a larger group specified by the dictionary key corresponding to the list

We'll use this helper function to make the replacements in order to consolidate the unique values in all required columns into larger groups

In [ ]:
for column, dict_replacements in col_replacements.items():
    replace_col_values(df=df, col2map=column, replacement_dict=dict_replacements)
    display(pd.DataFrame(df[column].value_counts(normalize=True)))

As we can see, the resulting number of unique values for each column that was mapped is smaller than that in the raw data, as expected. The number of observations in each of the newly assigned groups will not vary as much as in the original groupings. [One-hot encoding](https://en.wikipedia.org/wiki/One-hot) will now generate a smaller number of features than it would on the raw data.

In [ ]:
plot_horiz_bar(
    df,
    col_name="primary_type",
    ptitle="Classes of Crime (categorized)",
    xspacer=0.01,
    yspacer=0.4,
    fig_size=(8, 3),
    savefig=figs_dir_path,
    save_pref=save_pref,
)

In [ ]:
display(
    pd.DataFrame(df[nums + cats].isna().sum(), columns=["number_of_missing_values"])
)

<a id="pre-processing-categorical-features"></a>

## 5. [Pre-processing categorical features](#pre-processing-categorical-features)

In this section, we will perform one hot encoding on all selected categorical features in the `cats` list.

First, we will define variables for `X` (features) and `y` (class labels) to be used in modeling

In [ ]:
X = df.loc[:, nums + cats].copy()
y = df[target]
display(X.head())
pd.DataFrame(y).head()

Next, we'll calculate the number of unique one-hot encoded features, across all categorical features, that will be generated by this step

In [ ]:
expected_cats_cols = X[cats].apply(pd.Series.value_counts).count().sum()

Next, we will perform the one-hot encoding

In [ ]:
dummy = pd.get_dummies(X[cats].astype(str), prefix_sep="=")
X = pd.concat([X, dummy], axis=1)
X = X.drop(cats, axis=1)

Finally, we'll verify that the one-hot encoding has resulted in a `DataFrame` with the expected number of features calculated two steps above

In [ ]:
assert (
    X.shape[1] == len(nums) + expected_cats_cols
), "mismatch in expected number of OHE cols"

<a id="generate-training-and-testing-splits"></a>

## 6. [Generate training and testing splits](#generate-training-and-testing-splits)

Here, we will generate training, validation and testing splits of data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=2
)

In [ ]:
print(X_test.shape[0], X_train.shape[0] + X_val.shape[0] + X_test.shape[0])

Next, we'll apply under-sampling, over-sampling or no re-sampling, as was specified earlier in the user inputs section

In [ ]:
X_train_resampled, y_train_resampled = apply_over_under_sampling(
    X_train=X_train, y_train=y_train, target_balance=target_balance,
)

We'll compare the number of rows in the testing, training and resampled training splits of the data

In [ ]:
print(X_test.shape[0], X_train.shape[0], X_train_resampled.shape[0])

<a id="pre-processing-numerical-features"></a>

## 7. [Pre-processing numerical features](#pre-processing-numerical-features)

Here, we will perform pre-processing needed for the modeling process. Since we previously did this for the categorical features (one-hot encoding of all features in the `cats` list), pre-processing will only be performed on the numerical features and will be done inside a `sklearn` pipeline object

In [ ]:
numericals_transformer = Pipeline(steps=[("scaler", StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, nums)], remainder="passthrough"
)

<a id="experiments-in-modeling"></a>

## 8. [Experiments in modeling](#experiments-in-modeling)

Next, we'll use various models to separately predict the class labels. Each such run will constitute a single experiment, based on the inputs specified earlier

First, we'll assemble lists of instantiated models (classifiers) and model names, including `sklearn`'s [`DummyClassifier`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.dummy) which will provide baseline performance against which the predictive ability of more sophisticated models will be compared

In [ ]:
dummy_classifiers = []
dummy_names = []
for strategy in dummy_strategies:
    dummy_classifiers.append(
        DummyClassifier(**model_params["DummyClassifier__" + strategy])
    )
    dummy_names.append(f"DummyClassifier__{strategy}")

sklearn_all_classifiers = {
    "LogisticRegression": LogisticRegression(**model_params["LogisticRegression"]),
    "LinearSVC": LinearSVC(**model_params["LinearSVC"]),
    "GaussianNB": GaussianNB(**model_params["GaussianNB"]),
    "RandomForestClassifier": RandomForestClassifier(
        **model_params["RandomForestClassifier"]
    ),
}

models = dummy_classifiers + [
    sklearn_all_classifiers[k] for k in sklearn_wanted_classifiers
]
model_names = dummy_names + [type(model).__name__ for model in models[3:]]

Next, we'll use two helper functions to perform [KFCV](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation) using a single model on training data, and return training and validation scores

We'll also use a helper function to calclate the class-based prediction accuracy from a [normalized confusion matrix](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#confusion-matrix)

Next, we'll use each model from the above list to

1. perform KFCV on training data (if possible)
2. predict+score on validation and testing data
3. generate evaluations for each model's performance on the out-of-sample (testing) data, with results
   - stored in a DataFrame
   - visualized in plots
4. Plot all KFCV scores on single box plot, for all models
5. Print a normalized confusion matrix

All the above functionality will be wrapped into a helper function. This will serve as a comparison of multiple models' performance at predicting the type (category) of crime.

In [ ]:
cell_st = time()
print(f"Cell execution start time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

df_sc, df_sc_all, pipes_all = sklearn_trials(
    X_train=X_train,
    y_train=y_train,
    X_train_resampled=X_train_resampled,
    y_train_resampled=y_train_resampled,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    preprocessor=preprocessor,
    nums=nums,
    scoring_metric=scoring_metric,
    models=models,
    model_names=model_names,
    sk_folds=sk_folds,
    target_balance=target_balance,
    dummy_names=dummy_names,
    show_diagnostic_plots=False,
    figs_dir_path=figs_dir_path,
)

print(f"\nCell execution end time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

<a id="gradient-boosting-trials"></a>

## 9. [Gradient Boosting trials](#gradient-boosting-trials)

Next, we will *explore* the capability of using [Gradient Boosting](https://en.wikipedia.org/wiki/Gradient_boosting) on this dataset.

We'll try out the [`xgboost`](https://xgboost.readthedocs.io/en/latest/index.html#) and [`lightgbm`](https://github.com/microsoft/LightGBM) open-source libraries here to compute `f1_score`s and compare to the other machine learning models.

First, we'll use a helper function to train `xgboost` on training data and evaluate on validation data (starting from the one-hot encoded version of the features `X` and labels `y`). We will perform the following
1. generate train-validation-test splits
2. encode the class labels using `LabelEncoder`
3. train the model using the training split
4. Get the score on validation and testing splits

Next, we'll use this helper function to
1. train the model on the training data and provide the above-created validation set for model evaluation using the `eval_set` model hyperparameter
2. calculate the average evaluation metric on the testing data
   - since the model was trained using `eval_set`, which was the validation set, we will **only** calculate the average score on the test split of the data


We will also generate some classification visualizations for the trained model, using testing data

In [ ]:
cell_st = time()
print(f"Cell execution start time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

if use_boosting_classifiers:
    df_sc, df_sc_all = boosting_trials(
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        X_test=X_test,
        y_test=y_test,
        boosting_classifier_names=boosting_classifier_names,
        scoring_metric=scoring_metric,
        df_sc=df_sc,
        df_sc_all=df_sc_all,
        show_diagnostic_plots=False,
        save_pref=save_pref,
    )

print(f"\nCell execution end time {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

<a id="plot-model-comparison"></a>

## 10. [Plot model comparison](#plot-model-comparison)

Here, we'll compare models used in a horizontal box plot
- red filled circles will show the score on out-of-sample data (that was not used to train the model)

In [ ]:
try:
    df_sc_all
except NameError as e:
    if "name 'df_sc_all' is not defined" not in str(e):
        raise
else:
    plot_horizontal_box_plots(
        df=df_sc_all[["validation_scores", "test_score", "model_name"]],
        x="validation_scores",
        y="model_name",
        x2="test_score",
        plot_title="Comparison of Classifiers",
        marker_size=8,
        scatter_marker_size=12,
        marker_linewidth=1,
        scatter_marker_linewidth=1,
        marker_color="white",
        scatter_marker_color="darkred",
        edge_color="black",
        scatter_edge_color="black",
        xlabel=f"{scoring_metric} Validation Scores",
        fig_size=(12, 9),
        savefig=figs_dir_path,
        save_pref=save_pref,
    )

<a id="export-experiment-outputs"></a>

## 11. [Export experiment outputs](#export-experiment-outputs)

We'll append some columns to the `DataFrame`s of averaged and separate scores in order to summarize the conditions of the experiment. We'll export each of these `DataFrame`s to a separate `*.csv` file.

In [ ]:
if cloud_data == "no":
    export_experiment_results(
        df_sc=df_sc,
        df_sc_all=df_sc_all,
        target_balance=target_balance,
        nums=nums,
        scoring_metric=scoring_metric,
        experiment_summary_data_path=experiment_summary_data_path,
        experiment_all_cv_data_path=experiment_all_cv_data_path,
    )

<a id="export-data-for-mapping"></a>

## 12. [Export data for mapping](#export-data-for-mapping)

Here, we'll use a helper function to retrieve the predicted label and prediction probability for the testing data, for a single user-specified model.

We'll choose a single model and generate a summary of predictions and prediction probabilities for that model

In [ ]:
def get_preds_probas(est, X_test, y_test, mapper_dict):
    """
    Get prediction probabilities (if available) or return true and predicted
    labels
    """
    df_preds = pd.DataFrame(est.predict(X_test), index=X_test.index)
    if hasattr(est.named_steps["clf"], "predict_proba"):
        # Get prediction probabilities (if available)
        df_probas = pd.DataFrame(est.predict_proba(X_test), index=X_test.index)

        # Append prediction and prediction probabilities
        df_summ = pd.concat([df_preds, df_probas], axis=1)
        df_summ.columns = ["predicted_label"] + [
            f"probability_of_{i}" for i in range(0, len(np.unique(y_test)))
        ]

        # Get label (class) with maximum prediction probability for each row
        df_summ["max_class_number_manually"] = df_probas.idxmax(axis=1)
        df_summ["probability_of_max_class"] = df_probas.max(axis=1)

        # Compare .predict_proba() and manually extracted prediction
        # probability
        if "Dummy" not in type(est.named_steps["clf"]).__name__:
            lhs = df_summ["max_class_number_manually"]
            rhs = df_summ["predicted_label"].replace(mapper_dict)
            assert (lhs == rhs).eq(True).all()
    else:
        df_summ = df_preds.copy()
    # Get true label
    df_summ.insert(0, "true_label", y_test)
    return df_summ

In [ ]:
best_pipe = get_best_pipe_by_model_name(
    pipes_list=pipes_all, clf_name="LogisticRegression"
)

df_summary = get_preds_probas(
    est=best_pipe,
    X_test=X_test,
    y_test=y_test,
    mapper_dict=dict(zip(sorted(y_test.unique()), range(y_test.nunique()))),
)
df_summary.head()

Next, we'll merge these two predicted features to the testing data. Finally, we'll group the merged testing data, with predictions and prediction probabilities, in order to provide just enough data to the plotting notebook and to the dashboard file, so as to improve their respective plotting responsiveness.

In [ ]:
if cloud_data == "no":
    export_mapping_data(
        df=df, X_test=X_test, df_summary=df_summary, d_mapping_specs=d_mapping_specs,
    )

In [ ]:
df_test = df.loc[X_test.index, nums + cats + [target]]
assert df_test.shape[0] == X_test.shape[0]

In [ ]:
df_test["primary_type_pred"] = best_pipe.predict(X_test)
df_test["match"] = df_test.apply(
    lambda x: x["primary_type"] in x["primary_type_pred"], axis=1
)
df_test.head()

In [ ]:
cols = [
    "location_description",
    "community_area",
    "domestic",
    "arrest",
    "is_weekend",
    "is_dark",
    "year",
    "month",
    "district",
    "primary_type",
]
xtick_angle = 15
inter_plot_vertical_space = 1
palette = "bright"
fig_size = (15, 30)

In [ ]:
fig, axs = plt.subplots(figsize=fig_size, nrows=len(cols), ncols=1)
plt.subplots_adjust(hspace=inter_plot_vertical_space)

for ax, col in zip(axs, cols):
    g = sns.barplot(
        x=col,
        y="latitude",
        data=df_test,
        estimator=np.ma.count,
        hue="match",
        palette=palette,
        ci=None,
        ax=ax,
    )
    ax.grid(True)
    ax.set_xticklabels(
        ax.get_xticklabels(), rotation=xtick_angle, horizontalalignment="right"
    )
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_title(
        f"Occurrences predicted correctly (True) and incorrectly (False), by {col.title().replace('_', ' ')}",
        fontweight="bold",
        loc="left",
    )
    ax.tick_params(axis="x", which="major", pad=-5)
    ax.tick_params(axis="y", which="major", pad=-5)
    ax.legend(
        loc="upper left",
        bbox_to_anchor=(1, 1),
        framealpha=0,
        handletextpad=0,
        borderaxespad=0,
    )

In [ ]:
if cloud_data == "no":
    show_confusion_matrix(
        est=best_pipe,
        X_train=X_train_resampled,
        y_train=y_train_resampled,
        X_test=X_test,
        y_test=y_test,
    )

<a id="examine-feature-importances"></a>

## 13. [Examine feature importances](#examine-feature-importances)

Finally, we'll train the best pipeline and use it to generate a plot of the feature importances or model coefficients, for the of `n` most important features

In [ ]:
cell_st = time()

dfi = get_feature_permutation_importances(
    best_pipe=best_pipe,
    X_train=X_train,
    X_test=X_test,
    y_test=y_test,
    fig_size=(30, 90),
    figs_dir_path=figs_dir_path,
    save_pref=save_pref,
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

**Interpretations and Discussion**

1. The four features that constitute the highest feature and permutation importances are
   - domestic
     - whether a crime involves domestic violence
   - arrest
     - whether a crime involves an arrest
   - location decription
     - the description of the location where the crime was committed
   - district
     - the Chicafo police district where the crime was committed
2. Here, we will focus on the most frequently occurring sub-categories within each of these features
   - The best model is relatively worse at making predictions where the crime did not involve domestic violence or an arrest. It shows greater accuracy at predicting where either of these are present and particularly so for crimes when the crime involves domestic violence. These two characteristics of recorded criminal events in this dataset have the highest feature and permutation importances.
   - Crimes whose locations are described as public accessible or private housing have the largest number of occurrences and mis-classifications. Crimes committed on property adjoining public and private areas, but labeled in the data as one or the other, could account for some of this difficulty in the model classifying them separately. Private housing crimes have a higher permutation importance than crimes committed in publicly accessibile places and both have very low feature importances compared to less frequent location descriptions. Two less frequently occurring descriptions of the location of a crime - **business (academic)** and **dining/shopping** - have the highest feature and permutation importances of crime location descriptions. The model shows the same or worse [see business (academic)] accuracy at predicting crimes with these less frequent descriptions of the location where the crime was committed.
   - [Police Districts](https://news.wttw.com/sites/default/files/Map%20of%20Chicago%20Police%20Districts%20and%20Beats.pdf) 1, 2, 3, 11, 12, 14, 18 and 19 have the highest occurrences of crime and also the largest number of mis-classifications by the best model. Simultaneously, these have the highest feature and permutation importances. These are neighboring districts within the North-Central part of the city. District 1 includes [Downtown Chicago](https://www.google.com/maps/place/Downtown+Chicago,+IL,+USA/@41.879198,-87.6389064,14z) and districts 2 and 3 are just south of Downtown. It is not surprising that the model subsequently struggles to classify crimes in the West Community Area (covering the center-west part of the city). Note that these districts are picked based on the total number of crimes committed and not based on a subset of crimes filtered by certain criteria. Future work should focus on improving model classification accuracy in these North-Central police districts.
3. The best model is relatively better at classifying crimes that involve property damage than it does for less frequent crime types - criminal disturbances and crimes involving violence to humans. Due to the [class imbalance](https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-30164-8_110) problem mentioned earlier, more than half of each of the latter two types are being wrongly classified as property damage (the majority class) as seen from the confusion matrix. Future work could look into a combination of re-sampling and additional features can lead to an improvement here.